In [1]:
%load_ext autoreload

import os
import sys
from dotenv import load_dotenv

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
if parent_dir not in sys.path:
  sys.path.insert(0, parent_dir)

load_dotenv() #import 

start_year = 2019
end_year = 2024

output_dir = "sec_data"
load_edgar_index = False
load_10k = False
load_13f = False




True

In [17]:
%autoreload 2

from utils.python_helpers import edgar_service
import pandas as pd
edgar = edgar_service.EdgarService("cmccullough", "cmccullough@microsoft.com")
if load_edgar_index:

  s_and_p = pd.read_csv("sec_data/s&p 500.csv")
  ref = pd.read_csv("sec_data/reference_data.csv")
  cikfilter1 = s_and_p["CIK"].astype(str)
  cikfilter2 = ref["cik"].astype(str)
  cikfilter = pd.concat([cikfilter1, cikfilter2]).to_list()
  edgar.parse_master_index_file(cik_filter=cikfilter, output_dir="sec_data")
# edgar.download_filings_for_form("10-K",)

In [3]:
if load_10k:
  from utils.python_helpers import form_10k_extractor
  extractor = form_10k_extractor.Form10kExtractor()

  extractor.process_directory_files("./sec_data/reference_data.csv", "./sec_data/10k-raw", "./sec_data/10k_extracted")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Processing: 1065280 - 2019-01-29 - 1065280_2019-01-29_000043
Processing: 1091667 - 2019-01-31 - 1091667_2019-01-31_000029
Processing: 1166691 - 2019-01-31 - 1166691_2019-01-31_000005
Processing: 1326801 - 2019-01-31 - 1326801_2019-01-31_000009
Processing: 908937 - 2019-01-30 - 908937_2019-01-30_000008
Processing: 1051512 - 2019-02-22 - 1051512_2019-02-22_000008
Processing: 1065280 - 2019-02-08 - 1065280_2019-02-08_000079
Extracting 10-K from ./sec_data/10k-raw\2019_02\1065280_2019-02-08_000079.txt
Processing: 1142417 - 2019-02-27 - 1142417_2019-02-27_004527
Processing: 1158324 - 2019-02-22 - 1158324_2019-02-22_000604
Processing: 1283699 - 2019-02-07 - 1283699_2019-02-07_000015
Processing: 1289945 - 2019-02-28 - 1289945_2019-02-28_000011
Processing: 1304421 - 2019-02-26 - 1304421_2019-02-26_001019
Processing: 1335258 - 2019-02-28 - 1335258_2019-02-28_000024
Processing: 1366868 - 2019-02-28 - 1366868_

In [4]:
%autoreload 2

from utils.python_helpers import form_13f_hr_extractor
form13fExtractor = form_13f_hr_extractor.Form13F_HR_Extractor()


form13fExtractor.process_directory_files("./sec_data/13F_HR-raw", "./sec_data/13f_hr.csv")

parsing ./sec_data/13F_HR-raw\2020_08\1065521_2020-08-17_004032.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-02_004743.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-02_004745.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-09_004850.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-09_004852.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-09_004854.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-09_004856.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-16_004934.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-16_004936.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-16_004939.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-16_004941.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-23_005137.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-23_005139.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_2020-10-23_005141.txt
parsing ./sec_data/13F_HR-raw\2020_10\1065521_20

0

In [46]:
%autoreload 2

from utils.python_helpers import finance_reports_service
import pandas as pd

company_cik_ticker_ref = pd.read_json("sec_data/sec_company_tickers.json")

finance_reports_service = finance_reports_service.FinanceReportsService("cmcullough", "cmccullough@microsoft.com", company_cik_ticker_ref, "sec_data")

msftInfo = finance_reports_service.get_current_market_data(789019)
msftInfo

789019
MSFT


,2024-06-30,2023-06-30,2022-06-30,2021-06-30
Tax Effect Of Unusual Items,-99918000.0,-2850000.0,43754000.0,180160797.164637
Tax Rate For Calcs,0.182,0.19,0.131,0.138266
Normalized EBITDA,133558000000.0,105155000000.0,99905000000.0,83831000000.0
Total Unusual Items,-549000000.0,-15000000.0,334000000.0,1303000000.0
Total Unusual Items Excluding Goodwill,-549000000.0,-15000000.0,334000000.0,1303000000.0
Net Income From Continuing Operation Net Minority Interest,88136000000.0,72361000000.0,72738000000.0,61271000000.0
Reconciled Depreciation,22287000000.0,13861000000.0,14460000000.0,11686000000.0
Reconciled Cost Of Revenue,74114000000.0,65863000000.0,62650000000.0,52232000000.0
EBITDA,133009000000.0,105140000000.0,100239000000.0,85134000000.0
EBIT,110722000000.0,91279000000.0,85779000000.0,73448000000.0


In [53]:
%autoreload 2

from utils.python_helpers import financial_modeling_service


financial_modeling_service.run()

Scenario: Base Case
Intrinsic Value per Share: $68.86
Premium/Discount: 698.76%



Scenario: Upside Case
Intrinsic Value per Share: $91.00
Premium/Discount: 504.39%



Scenario: Downside Case
Intrinsic Value per Share: $51.42
Premium/Discount: 969.53%



Scenario: Extreme Downside Case
Intrinsic Value per Share: $34.04
Premium/Discount: 1515.77%



                Scenario  Intrinsic Value per Share  Premium/Discount
0              Base Case                  68.856899          6.987580
1            Upside Case                  91.001573          5.043852
2          Downside Case                  51.424512          9.695289
3  Extreme Downside Case                  34.039466         15.157715


In [98]:
%autoreload 2

from utils.python_helpers.fin_models import subscription_based
import numpy as np
import plotly.express as px
years = np.array([2018, 2019, 2020, 2021, 2022])
users = np.array([500000, 750000, 1100000, 1600000, 2300000])
future_years = np.arange(2023, 2028)


[2996868.28344136 3718919.67527903 4362239.63940992 4878825.57481482
 5260106.09463992]
